In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB

In [2]:
df=pd.read_csv(r"C:\Users\karri\AI TASK-4\spam.csv")

In [3]:
df


,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [4]:
df.groupby('Category').count()

,Message
Category,
ham,4825
spam,747


In [5]:
cat=pd.get_dummies(df['Category'],drop_first=True)
cat.head()

,spam
0,0
1,0
2,1
3,0
4,0


In [6]:
df=pd.concat([df,cat],axis=1)

In [7]:
df.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [8]:
train_data = df.sample(frac=0.8,random_state=1).reset_index(drop=True)
test_data = df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)

In [9]:
df['Message'].value_counts() / df.shape[0] * 100

Sorry, I'll call later                                                                                                                                      0.538406
I cant pick the phone right now. Pls send a message                                                                                                         0.215363
Ok...                                                                                                                                                       0.179469
Ok                                                                                                                                                          0.071788
Ok.                                                                                                                                                         0.071788
                                                                                                                                                              ...   
I gotta co

In [10]:
train_data['Message'].value_counts() / train_data.shape[0] * 100

Sorry, I'll call later                                                                                                                                                 0.471063
I cant pick the phone right now. Pls send a message                                                                                                                    0.224316
Ok...                                                                                                                                                                  0.157021
Say this slowly.? GOD,I LOVE YOU &amp; I NEED YOU,CLEAN MY HEART WITH YOUR BLOOD.Send this to Ten special people &amp; u c miracle tomorrow, do it,pls,pls do it...    0.089726
7 wonders in My WORLD 7th You 6th Ur style 5th Ur smile 4th Ur Personality 3rd Ur Nature 2nd Ur SMS and 1st "Ur Lovely Friendship"... good morning dear                0.089726
                                                                                                                        

In [11]:
test_data['Message'].value_counts() / test_data.shape[0] * 100

Sorry, I'll call later                                                                                                                                            0.359066
I cant pick the phone right now. Pls send a message                                                                                                               0.269300
Oh k. . I will come tomorrow                                                                                                                                      0.179533
Gud ni8 dear..slp well..take care..swt dreams..Muah..                                                                                                             0.179533
SMS SERVICES. for your inclusive text credits, pls goto www.comuk.net login= 3qxj9 unsubscribe with STOP, no extra charge. help 08702840625.COMUK. 220-CM2 9AE    0.179533
                                                                                                                                                 

In [12]:
vocabulary = list(set(train_data['Message'].sum()))
word_counts_per_sms = pd.DataFrame([
    [row[1].count(word) for word in vocabulary]
    for _, row in train_data.iterrows()], columns=vocabulary)
train_data = pd.concat([train_data.reset_index(), word_counts_per_sms], axis=1).iloc[:,1:]

In [13]:
train_data.head()

,Category,Message,spam,>,"""",z,o,I,6,0,...,C,.,x,Q,Y,5,〨,|,),<
0,ham,"Yep, by the pretty sculpture",0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,ham,"Yes, princess. Are you going to make me moan?",0,0,0,0,4,0,0,0,...,0,1,0,0,1,0,0,0,0,0
2,ham,Welp apparently he retired,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,Havent.,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,ham,I forgot 2 ask ü all smth.. There's a card on ...,0,0,0,0,6,1,0,0,...,0,5,0,0,0,0,0,0,0,0


In [14]:
Pspam = train_data['Category'].value_counts()['spam'] / train_data.shape[0]

In [15]:
Pham = train_data['Category'].value_counts()['ham'] / train_data.shape[0]

In [16]:
Nspam = train_data.loc[train_data['Category'] == 'spam', 
                       'Message'].apply(len).sum()
# NUmber of words in Spam message

In [17]:
Nham = train_data.loc[train_data['Category'] == 'ham', 
                       'Message'].apply(len).sum()

In [18]:
Nvoc = len(train_data.columns)-3

In [19]:
def p_w_spam(word):
    if word in train_data.columns:
        return (train_data.loc[train_data['Category'] == 'spam', word].sum() + alpha) / (Nspam + alpha*Nvoc)
    else:
        return 1
      
def p_w_ham(word):
    if word in train_data.columns:
        return (train_data.loc[train_data['Category'] == 'ham', word].sum() + alpha) / (Nham + alpha*Nvoc)
    else:
        return 1

In [20]:
def classify(Message):
    p_spam_given_message = Pspam
    p_ham_given_message = Pham
    for word in message:
        p_spam_given_message *= p_w_spam(word)
        p_ham_given_message *= p_w_ham(word)
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [21]:
df


,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,1
5568,ham,Will ü b going to esplanade fr home?,0
5569,ham,"Pity, * was in mood for that. So...any other s...",0
5570,ham,The guy did some bitching but I acted like i'd...,0
